# *Growth Profiler* Data Analysis

The *Growth Profiler* is a high throughput device for growth characterization. Growth is automatically measured via OD in 96-well plates and the results are stored in csv files. This workflow trains analysis of growth data in csv format.

## Loading libraries
There are some general Python libraries necessary in order to have usefull functions available. The library responsible for growth curve analysis is `batchslopes` [Link](https://github.com/uliebal/gp_analytics) and contains functions specifically designed for analysis of the *Growth Profiler*. 

In [ ]:
import numpy as np
import pandas as pd
import os
import warnings
# from batchslopes import *
# import sys
# sys.path.append('..')
!pip install batchslopes
from batchslopes import *
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
# from collections import OrderedDict
import matplotlib.pyplot as plt

## Data input and descripion

The standard growth profiler csv file contains in the first rows some general information on the experimental conditions. We ignore these metadata in the regression analysis, however, make sure to include the header names when the real data starts. The first column should contain the time measurements, and all other columns are OD values. The *Growth Profiler* usually measures the time in minutes, other experimental data, like in *RecExpSim*, uses hours as time unit. The workflow tries to unify the time-based analysis by dividing the time vector with the variable `TimeUnit`.

The data is partitioned (binned, variable `Partition`) and in each bin a regression on the logarithmic data is conducted. The bin with the best correlation coefficient is then selected for further partitioning until the correlation coefficient gets worse (is decreasing). The correlation coefficient and the corresponding slope of the logarithmic OD are reported.

**Important**: Check the decimal separator. Science typically works with the english convention of point-separators (`10,000.23`), but the *Growth Profiler* or your local system might generate an csv with german comma-separators (`10.000,23`).


**Input:**
 - `File`: string, growth experiment csv-file name
 - `skiprows`: integer, lines of experimental metadata, Start counting from 0.
 - `decimal`: string ('.' or ','), decimal separator 
 - `TimeUnit`: integer, for measure time in minutes: 60, hours: 1
 - `Partition`: integer, decides the number of bins on which regression is performed

In [ ]:
File = None
skiprows = None
decimal = None
TimeUnit = None
Partition = None
df = pd.read_csv(File, skiprows=skiprows , decimal=decimal)
df = df.drop(labels='Input_Image', axis=1)
TimeAx = df.columns[0] #'Time (min)' # for growth profiler
Exp0 = df.columns[1]
print('Column headers: ', df.columns)
df.plot(x=TimeAx, y=Exp0)

## Single Experiment Analysis

Below, we check individual growth curves and their regression. This can be helpful if you want to examine a particular experiment.

In [ ]:
t = df[TimeAx].values/TimeUnit
x = df[Exp0].values
myResult = DetectR2MaxSingle(t,x,Partition)
print(myResult)
plt.plot(t, np.log(x))
if myResult!=False:
    plt.scatter(myResult['time'], np.log(myResult['OD']))
    plt.plot(myResult['time'], myResult['Slope']*myResult['time'] + myResult['ycorrect'], 'r', label='$\mu$:{:.3f}, R2:{:.2f}'.format(myResult['Slope'], myResult['R2']))
    plt.legend()
    myResult['ID'] = Exp0

# myResult

## All Experiment Analysis

Below, all experiments in the csv file are analysed. The final information is stored in plots showing the regression range for each experiment along with the growth rate and the correlation coefficient and stored as `myPlots.svg`. If the regressions look unconvincing, try a different partition.

In [ ]:
myCols = df.columns[1:-2]
NumExp = len(myCols)
AxDim = np.ceil(np.sqrt(NumExp))
t = df[TimeAx].values/TimeUnit
# plt.subplots(AxDim, AxDim, sharex='col')
fig, ax = plt.subplots(figsize=[20,10], sharey=True)
for idx, myExp in enumerate(myCols):
#     print(myExp)
    x = df[myExp].values
    myResult = DetectR2MaxSingle(t,x,Partition)
    plt.subplot(5,5, idx+1)
    plt.plot(t, np.log(x))
    plt.title(myExp)
    if myResult is not False:
        plt.scatter(myResult['time'], np.log(myResult['OD']))
        plt.plot(myResult['time'], myResult['Slope']*myResult['time'] + myResult['ycorrect'], 'r', label='$\mu$:{:.3f}, R2:{:.2f}'.format(myResult['Slope'], myResult['R2']))
        myResult['ID'] = myExp
        plt.legend()
    plt.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
    
fig.text(0.5, 0.1, 'time', ha='center')
fig.text(0.1, 0.5, 'ln(OD)', va='center', rotation='vertical')
plt.savefig('myPlots.svg', format='svg')